<a href="https://colab.research.google.com/github/nsi319/Reformers/blob/main/reformer_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi
!pip install -U reformer_pytorch
!pip install -U transformers

Fri Mar  5 15:53:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget http://www.gutenberg.org/files/2554/2554-0.txt

--2021-03-04 06:15:29--  http://www.gutenberg.org/files/2554/2554-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1201735 (1.1M) [text/plain]
Saving to: ‘2554-0.txt’

2554-0.txt          100%[===================>]   1.15M  1.47MB/s    in 0.8s    

2021-03-04 06:15:30 (1.47 MB/s) - ‘2554-0.txt’ saved [1201735/1201735]



In [ ]:
import torch

from reformer_pytorch import ReformerEncDec
from datetime import datetime

from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("google/reformer-crime-and-punishment")

# text = """On March 2, 2018, the Securities and Exchange Commission announced securities fraud charges against a U.K.-based broker-dealer and its investment manager in connection with manipulative trading in the securities of HD View 360 Inc., a U.S.-based microcap issuer.  The SEC also announced charges against HD View's CEO, another individual, and three entities they control for manipulating HD View's securities as well as the securities of another microcap issuer, West Coast Ventures Group Corp.  The SEC further announced the institution of an order suspending trading in the securities of HD View.These charges arise in part from an undercover operation by the Federal Bureau of Investigation, which also resulted in related criminal prosecutions against these defendants by the Office of the United States Attorney for the Eastern District of New York.In a complaint filed in the U.S. District Court for the Eastern District of New York, the SEC alleges that Beaufort Securities Ltd. and Peter Kyriacou, an investment manager at Beaufort, manipulated the market for HD View's common stock.  The scheme involved an undercover FBI agent who described his business as manipulating U.S. stocks through pump-and-dump schemes.  Kyriacou and the agent discussed depositing large blocks of microcap stock in Beaufort accounts, driving up the price of the stock through promotions, manipulating the stock's price and volume through matched trades, and then selling the shares for a large profit.The SEC's complaint against Beaufort and Kyriacou alleges that they:opened brokerage accounts for the undercover agent in the names of nominees in order to conceal his identity and his connection to the anticipated trading activity in the accounts suggested that the undercover agent could create the false appearance that HD View's stock was liquid in advance of a pump-and-dump by "gam[ing] the market" through matched trades executed multiple purchase orders of HD View shares with the understanding that Beaufort's client had arranged for an associate to simultaneously offer an equivalent number of shares at the same priceA second complaint filed by the SEC in the U.S. District Court for the Eastern District of New York alleges that in a series of recorded telephone conversations with the undercover agent, HD View CEO Dennis Mancino and William T. Hirschy agreed to manipulate HD View's common stock by using the agent's network of brokers to generate fraudulent retail demand for the stock in exchange for a kickback from the trading proceeds.  According to the complaint, the three men agreed that Mancino and Hirschy would manipulate HD View stock to a higher price before using the agent's brokers to liquidate their positions at an artificially inflated price.  The SEC's complaint also alleges that Mancino and Hirschy executed a "test trade" on Jan. 31, 2018, coordinated by the agent, consisting of a sell order placed by the defendants filled by an opposing purchase order placed by a broker into an account at Beaufort.  Unbeknownst to Mancino and Hirschy, the Beaufort account used for this trade was a nominal account that was opened and funded by the agent.  The SEC's complaint also alleges that, prior to their contact with the undercover agent, Mancino and Hirschy manipulated the market for HD View and for West Coast by using brokerage accounts that they owned, controlled, or were associated with –including TJM Investments Inc., DJK Investments 10 Inc., WT Consulting Group LLC – to effect manipulative "matched trades."The SEC's complaint against Beaufort and Kyriacou charges the defendants with violating Section 10(b) of the Securities Exchange Act of 1934 and Rule 10b-5 thereunder.  The SEC also charged Hirschy, Mancino, and their corporate entities with violating Section 17(a)(1) of the Securities Act of 1933, Sections 9(a)(1), 9(a)(2), and 10(b) of the Exchange Act and Rules 10b-5(a) and (c) thereunder.  The SEC is seeking injunctions, disgorgement, prejudgment interest, penalties, and penny stock bars from Beaufort and Kyriacou.  With respect to Hirschy, Mancino, and their corporate entities, the SEC is seeking injunctions, disgorgement, prejudgment interest, penalties, penny stock bars, and an officer-and-director bar against Mancino.The investigation was conducted in the SEC's New York Regional Office by Tejal Shah and Joseph Darragh, Lorraine Collazo, and Michael D. Paley of the Microcap Fraud Task Force and supervised by Lara S. Mehraban, and in Washington, D.C. by Patrick L. Feeney, Robert Nesbitt, and Kevin Guerrero, and supervised by Antonia Chion.  Preethi Krishnamurthy and Ms. Shah will lead the SEC's litigation against Beaufort and Kyriacou.  Ann H. Petalas and Mr. Feeney, under the supervision of Cheryl Crumpton, will handle the SEC's litigation against Mancino, Hirschy, and their entities.  The SEC appreciates the assistance of the Office of the United States Attorney for the Eastern District of New York, the Federal Bureau of Investigation, the Internal Revenue Service, the Alberta Securities Commission, the Ontario Securities Commission, the Financial Conduct Authority of the United Kingdom, and the Financial Industry Regulatory Authority.The Commission's investigation in this matter is continuing."""
# summ = """On March 2, 2018, the Securities and Exchange Commission charged Beaufort Securities Ltd. and Peter Kyriacou, an investment manager at Beaufort, with manipulating the market for HD View 360 Inc., a U.S.-based microcap issuer.  The SEC also announced charges against HD View's CEO, another individual, and three entities they control for manipulating HD View through pump-and-dump schemes.  According to the SEC's complaint, the defendants discussed depositing large blocks of microcap stock in Beaufort accounts, driving up the price of the stock through promotions, manipulating the stock's price and volume through matched trades, and then selling the shares for a large profit.  In a parallel action, the United States Attorney's Office for the Eastern District of New York announced criminal charges against the defendants.  On March 4, the SEC announced the entry of an order suspending trading in the securities of HD View and for West Coast, pending the outcome of a parallel criminal action by the Federal Bureau of Investigation.  Following the announcement of the suspension, HD View stock prices and volume increased significantly, and the defendants agreed to pay over $1.5 million in disgorgement, prejudgment interest, penalties, and an officer and director bar.  Beaufort agreed to settle the charges without admitting or denying the allegations of the complaint, and to pay a $1 million civil penalty. The SEC's investigation, which is continuing, has been conducted by Patrick McCluskey and Cheryl Crumpton of the SEC Enforcement Division's Market Abuse Unit in the New York Regional Office.Â  The SEC appreciates the assistance of the Financial Industry Regulatory Authority of the United Kingdom, the Canadian Securities Commission, the Alberta Securities Commission and the Ontario Securities Commission."""

text = """He had shown signs of some obscure nervous disease before his arrest and this now developed into violent attacks of epilepsy, from which he suffered for the rest of his life. The fits occurred three or four times a year and were more frequent in periods of great strain. In 1859 he was allowed to return to Russia. He started a journal--“Vremya,” which was forbidden by the Censorship through a misunderstanding. In 1864 he lost his first wife and his brother Mihail. He was in terrible poverty, yet he took upon himself the payment of his brother’s debts. He started another journal--“The Epoch,” which within a few months was also prohibited. He was weighed down by debt, his brother’s family was dependent on him, he was forced to write at heart-breaking speed, and is said never to have corrected his work. The later years of his life were much softened by the tenderness and devotion of his second wife."""
summ = """In 1864 he lost his first wife and his brother Mihail. He was in terrible poverty, yet he took upon himself the payment of his brother’s debts. He was weighed down by debt, his brother’s family was dependent on him, he was forced to write at heart-breaking speed, and is said never to have corrected his work.""" 


In [ ]:
CUDA_LAUNCH_BLOCKING=1
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
input_ids = tokenizer.encode(text, return_tensors='pt').long().cuda()
output_ids = tokenizer.encode(summ, return_tensors="pt").long().cuda()

print(input_ids)
print("Input shape: ", input_ids.shape)
print("Output shape: ", output_ids.shape)

t1 = datetime.now()
print("Start Time: ", t1)
DE_SEQ_LEN = 2048
EN_SEQ_LEN = 2048

enc_dec = ReformerEncDec(
    dim = 512,
    enc_num_tokens = 60000,
    enc_depth = 4,
    enc_max_seq_len = DE_SEQ_LEN,
    dec_num_tokens = 60000,
    dec_depth = 4,
    dec_max_seq_len = EN_SEQ_LEN
).cuda()

t2 = datetime.now()
print("Time taken to load model (in seconds): ",(t2-t1).total_seconds())
# train_seq_in = torch.randint(0, 64000, (1, DE_SEQ_LEN)).long().cuda()
# train_seq_out = torch.randint(0, 64000, (1, EN_SEQ_LEN)).long().cuda()

train_seq_in = input_ids
train_seq_out = output_ids
input_mask = torch.ones(1, DE_SEQ_LEN).bool().cuda()

loss = enc_dec(train_seq_in, train_seq_out, return_loss = True, enc_input_mask = input_mask)
print("Loss:",loss)
loss.backward()
# learn

t3 = datetime.now()
print("Time taken to train the model (in seconds): ",(t3-t2).total_seconds())
# evaluate with the following
eval_seq_in = torch.randint(0, 310, (1, DE_SEQ_LEN)).long().cuda()
eval_seq_out_start = torch.tensor([[0.]]).long().cuda() # assume 0 is id of start token
samples = enc_dec.generate(input_ids, eval_seq_out_start, seq_len = EN_SEQ_LEN, eos_token = 1) # assume 1 is id of stop token

print(samples.shape) # (1, <= 1024) decode the tokens
print("generated output: ", samples)
summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in samples][0]


print("Decoded text: ")
print(summary)
t4 = datetime.now()
print("Time taken to generate (in seconds): ",(t4-t3).total_seconds())

tensor([[126, 100, 168,  77, 263,   7, 264, 275, 263, 266,  44, 240,  14, 280,
         266, 274, 270,  10, 136,  16, 281, 190,  30,  24, 259,  29, 259,  49,
         276, 144,  80,   4, 267,  10,  63,  27, 221, 254, 166,  57, 269, 262,
         279,  19, 199, 262, 258, 281, 264, 112,  76,  78, 260, 261, 134, 266,
          44,  64, 279, 264,  58, 279, 266, 272, 277, 188, 114, 264,  86,  23,
         117, 276, 276, 101, 268,  88,  13,  87,  63,  44,  80,  35, 141, 259,
         278, 140,  22,  15, 266,  14, 274, 274, 133,  10, 268,  65,  10, 259,
         253,  22, 170,   3,  51,  42,   4,  36, 259,  52,  27, 184,  18, 144,
          22,  10, 179,  76,  39,  40,  16, 264, 227, 266,  44,  54,  10,  50,
          74, 267, 124, 278,  33, 263, 258,   0,  23,  59, 120,  77,  19,  26,
          87, 260, 133, 263,  26,  98, 270, 173, 143, 278, 126,  74, 252,  19,
           4, 258, 304, 170, 263,  94, 197, 285,   0,  10, 271, 272, 261, 135,
         114, 264,  86,  59,  88, 280,  62, 268,  25

AssertionError: ignored

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 17.0MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("google/reformer-crime-and-punishment")

In [ ]:
print(len(tokenizer))
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens_extended)

print(tokenizer.encode("Hello I am naren. This is a new naren. hhh"))

320
[2, 0]
['</s>', '<unk>']
[126, 32, 262, 33, 207, 136, 251, 263, 278, 108, 265, 24, 111, 4, 232, 273, 136, 251, 263, 278, 31, 265, 265]


In [ ]:
 
from reformer_pytorch import ReformerLM
from reformer_pytorch.generative_tools import TrainingWrapper

import random
import tqdm
import gzip
import numpy as np
import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from datetime import datetime


# constants

NUM_BATCHES = int(100)
BATCH_SIZE = 4
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 1e-4
VALIDATE_EVERY  = 10
GENERATE_EVERY  = 20
GENERATE_LENGTH = 300
SEQ_LEN = 1024

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data


# instantiate model

model = ReformerLM(
    dim = 512,
    depth = 6,
    max_seq_len = 1024,
    num_tokens = len(tokenizer),
    heads = 8,
    bucket_size = 64,
    n_hashes = 4,
    ff_chunks = 10,
    lsh_dropout = 0.1,
    weight_tie = True,
    causal = True,
    n_local_attn_heads = 4,
    use_full_attn = False # set this to true for comparison with full attention
)

model = TrainingWrapper(model)
model.cuda()

# prepare enwik8 data

with open('/content/2554-0.txt') as file:
    data = file.read()
    X = tokenizer.encode(data, return_tensors="pt")[0]
    print("\n")
    print(X)
    print(len(X))
    trX, vaX = np.split(X, [int(30000)])
    data_train, data_val = trX,vaX

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        full_seq = self.data[:self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# training

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss = model(next(train_loader), return_loss = True)
        loss.backward()

    print(f'\n training loss: {loss.item()}')
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            loss = model(next(val_loader), return_loss = True)
            print(f'\nvalidation loss: {loss.item()}')
            

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        # prime = tokenizer.decode(inp, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        # print(f'\n', (prime))
        t1 = datetime.now()
        sample = model.generate(inp, GENERATE_LENGTH)
        t2 = datetime.now()
        print("\nTime taken to generate for {} is {} seconds".format(i,t2-t1))
        output_str = tokenizer.decode(sample, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        print("\n Generated sample: ", output_str)

Token indices sequence length is longer than the specified maximum sequence length for this model (526488 > 524288). Running this sequence through the model will result in indexing errors
training:   0%|          | 0/100 [00:00<?, ?it/s]



tensor([140, 125, 186,  ..., 266, 278, 258])
526488

 training loss: 5.9030656814575195

validation loss: 5.887028217315674


training:   1%|          | 1/100 [00:12<20:10, 12.22s/it]


Time taken to generate for 0 is 0:00:10.438608 seconds

 Generated sample:  veppriaskolnikovutEEilhin h h with ag didookasYou thisus(YouE in:s “ wereityad upon ag connikov wouldblequ her lookil or there Hebleyil Hexss yournikov hhinooadj h worould there looketadet<s> He wouldid or h But yourYou that there h with by youril p And yjould wor that hight The int(j is yourble y conblej conss now agadE would notid oradellght y wouldndhinYouGamehinllhileroo didad someooadhinellbleoutyould wouldnikov hidquC:YouEid isble wouldadjutousE or in Raskolnikov didss there Raskolnikov h yad somezzhinzour un_ in Butoutpp youroo thereYou oromeroourEionsx h thatble yveet And her yourhin lookilgble(id woradurnd_adoneYou “g y con conE cony nowzouldadble RaskolnikovveryEal unour heuroneri upYou( yisilet: Theould wor(icjvej your your yourilYouRadationYou would orhaetomej in worilYou or your yourhinou wornikov yxouldjysenikoviletot her: frstooalE y thereEau

 training loss: 5.777960777282715

 training loss: 5

training:   8%|▊         | 8/100 [00:22<13:48,  9.00s/it]

...


 training loss: 0.25379714369773865

 training loss: 0.23526564240455627

 training loss: 0.21676293015480042

 training loss: 0.19972017407417297

validation loss: 6.174382209777832

 training loss: 0.18375617265701294

 training loss: 0.16971945762634277

 training loss: 0.15591777861118317


training:  95%|█████████▌| 95/100 [03:19<00:10,  2.00s/it]


 training loss: 0.14277097582817078

 training loss: 0.13101473450660706

 training loss: 0.11997829377651215

 training loss: 0.10992483049631119

 training loss: 0.10051631182432175


training: 100%|██████████| 100/100 [03:26<00:00,  2.07s/it]


 training loss: 0.09182821959257126


In [ ]:
text = "A few months later"
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
t1 = datetime.now()
output_ids = model.generate(input_ids,100)
t2 = datetime.now()
print("\nTime taken to generate is {} seconds".format(t2-t1))
print(output_ids)
output_str = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("Generated sample: ", output_str)


Time taken to generate is 0:00:03.495704 seconds
tensor([[275, 258, 317, 300, 158,  44,  13, 258, 314,  67, 109, 188, 187, 272,
         259,  58, 277, 242, 258, 313, 272,  98, 272, 227,  38, 216, 262,  63,
         262, 108,  51, 259,  76,  38, 272, 108, 265,  24, 265,  24, 265,  24,
         265, 271,  26,  13, 125, 186, 272, 276, 264, 134,  48, 311,  27,  84,
         271, 262,  63,  27,  84, 174,   9,  41, 260, 265,  87,  63,  82, 123,
          92, 266, 178, 266, 262, 259,  89,  27, 166, 193,  53,  21, 265,   9,
          66, 186,  56, 277,  54,   3, 259, 271, 266,  44, 253,  87, 289, 270,
          70,  56]], device='cuda:0')
Generated sample:  g EBook of the Cetter from Byele, by Fy Ryodor Dosto Timeentory Thishishishm to the ProyficklyK and withmost and with alou youth restrictions whatsoever and de ex no choughro it, g tems of or re-use it


In [ ]:
google = AutoModelWithLMHead.from_pretrained("google/reformer-crime-and-punishment")
google_tokenizer = AutoTokenizer.from_pretrained("google/reformer-crime-and-punishment")
print(google_tokenizer.decode(google.generate(google_tokenizer.encode("A few months later", return_tensors="pt"), do_sample=True,temperature=0.7, max_length=100)[0]))

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of ReformerModelWithLMHead were not initialized from the model checkpoint at google/reformer-crime-and-punishment and are newly initialized: ['reformer.encoder.layers.0.attention.self_attention.mask_value_float16', 'reformer.encoder.layers.0.attention.self_attention.mask_value_float32', 'reformer.encoder.layers.1.attention.self_attention.self_mask_value_float16', 'reformer.encoder.layers.1.attention.self_attention.self_mask_value_float32', 'reformer.encoder.layers.1.attention.self_attention.mask_value_float16', 'reformer.encoder.layers.1.attention.self_attention.mask_value_float32', 'reformer.encoder.layers

A few months later, but she was only the old woman, but her little. “Here!” she cried, laughing, trying to figure, drawinging her head head. He laid time he turned to the other. Marmeladov was in the new had not


In [ ]:
text = "A few months later"
input_ids = google_tokenizer.encode(text, return_tensors="pt").cuda()
t1 = datetime.now()
output_ids = model.generate(input_ids,100)
t2 = datetime.now()
print("\nTime taken to generate is {} seconds".format(t2-t1))
print(output_ids)
output_str = google_tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("Generated sample: ", output_str)


Time taken to generate is 0:00:03.428068 seconds
tensor([[28515, 30590,   173, 43251,  5375,  3872,  1268,     8,  6203,    24,
             6,    49,   367, 13617, 50118,   133, 42735, 50118,     5,   274,
           219, 28145,   211,  2603,   139,  3623, 18708, 50118,  3623, 18708,
         50118,   163,   756,  2226, 23985,   492,   117,    42,  8793,     6,
           492,    24, 21979,   370,    24,  3037,    38,    21,  1166,    27,
           870, 13593,  3486,    78,   220, 22401,  2124,     5, 50118, 23375,
             5, 50118,  4283,   523,    30,   219, 28145,  6372,    51,   779,
          6372,  1519,  3503, 27614, 29183,     6, 24751,   534,    35,    58,
         12613,   336, 50118,   250,     9,     5, 45326,  3885,     6,     5,
         50118, 50118,   849, 50140, 50118, 50118, 42779,  1957, 13735, 50118]],
       device='cuda:0')
Generated sample:  ss wisin beurhecusive?”na suesinisisor c and at theyhehe dusimin lle saidouheinhe
